In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
import time, sys, os

directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
plt.rcParams['figure.dpi']=400

# Implementation of the TEBD for a test case

## Assignment: MPS simulations using TEBD

We will again study the quantum Ising model subject to a magnetic field with both transverse and longitudinal components:

$$
H = -J \sum_{j=1}^{L-1} \sigma_j^z \sigma_{j+1}^z - h^x \sum_{j=1}^{L} \sigma_j^x - h^z \sum_{j=1}^{L} \sigma_j^z .
$$

Set $ J = 1 $ and the magnetic field to $ (h^x, h^z) = (-1.05, 0.5) $, the same values as in Assignment 3, to allow you to test your MPS-based approach against exact diagonalization for small system sizes. Because we are now working with MPS, consider the system with open boundary conditions where the MPS simulations are most transparent. To specify a time-evolution procedure, this Hamiltonian can be arranged into three groups of commuting terms (i.e., commuting within each group): $ H = H_{\text{odd}} + H_{\text{even}} + H_{\text{field}} $, each of which can be readily exponentiated. The groupings are:

$$
H_{\text{odd}} = -J \sum_{j=1,3,5,\ldots} \sigma_j^z \sigma_{j+1}^z = -J (\sigma_1^z \sigma_2^z + \sigma_3^z \sigma_4^z + \sigma_5^z \sigma_6^z + \cdots ) ,
$$

$$
H_{\text{even}} = -J \sum_{j=2,4,6,\ldots} \sigma_j^z \sigma_{j+1}^z = -J (\sigma_2^z \sigma_3^z + \sigma_4^z \sigma_5^z + \sigma_6^z \sigma_7^z + \cdots ) ,
$$

$$
H_{\text{field}} = \sum_{j=1}^{L} (-h^x \sigma_j^x - h^z \sigma_j^z) = -h^x \sigma_1^x - h^z \sigma_1^z - h^x \sigma_2^x - h^z \sigma_2^z - h^x \sigma_3^x - h^z \sigma_3^z - \cdots .
$$

Clearly the terms in $ H_{\text{odd}} $ commute, and similarly for $ H_{\text{even}} $, so they can be exponentiated directly:

$$
e^{-itH_{\text{odd}}} = e^{itJ\sigma_1^z \sigma_2^z} e^{itJ\sigma_3^z \sigma_4^z} e^{itJ\sigma_5^z \sigma_6^z} \cdots , \quad e^{-itH_{\text{even}}} = e^{itJ\sigma_2^z \sigma_3^z} e^{itJ\sigma_4^z \sigma_5^z} e^{itJ\sigma_6^z \sigma_7^z} \cdots .
$$

Within $ H_{\text{field}} $, $ \sigma_j^x $ and $ \sigma_j^z $ do not commute on the same site. However, as these are single-site operators we can combine the terms for each $ j $ into

$$
\omega_j \equiv -h^x \sigma_j^x - h^z \sigma_j^z = \begin{bmatrix} -h^z & -h^x \\ -h^x & h^z \end{bmatrix} ,
$$

written in the $ \sigma^z $ basis. As all of the $ \omega_j $ commute, now $ e^{-itH_{\text{field}}} = e^{-it\omega_1} e^{-it\omega_2} e^{-it\omega_3} \cdots $. Each $ e^{-it\omega_j} $ can be written out using formulas for Pauli matrices, e.g., $ \exp(i\phi \mathbf{n} \cdot \sigma) = \cos(\phi) + i \sin(\phi) \mathbf{n} \cdot \sigma $ [where $ \mathbf{n} $ is a unit vector] for real time evolution, substituting hyperbolic functions for imaginary time evolution, or by direct exponentiation of the matrix. (Incidentally, you may notice that in this case other Trotter patterns than the one presented below are possible and may be more efficient. If you'd like, you can explore some of these, but the scheme outlined above will work regardless of the details of the terms in $ H $.)

### 4.1 Imaginary time evolution

"Rotate" now to imaginary time $\tau = it$ and perform TEBD for cooling to the ground state. It will turn out that in this case all the tensors are real-valued, but you should write your solution to also handle complex-valued tensors, for example using actual Hermitian conjugates rather than transposes; this will greatly simplify the process of going to real time evolution. However if your programming language is not strict about types, you may need to periodically cast complex values to reals in order to use specialized linear algebra routines.


## Test wave-function

In [178]:
def ferromagnetic_state_MPS(L):
    
    # given L, returns the MPS tensors of the ferromagnetic state.
    
    mps_tensors= []
    
    A = np.zeros((1, 2, 1), dtype = complex)
    A[0, 0, 0] = 1
    A[0, 1, 0] = 0
    
    for i in range(L):
        mps_tensors.append(A)

    return mps_tensors

## Useful troubleshooting code

In [126]:
def mps(psi, k):

    # Converts a wavefunction into mps form
    
    mps_tensors = []
    L = int(np.log2(len(psi)))  # Number of spins/sites
    psi_matrix = psi.reshape(2, 2**(L-1))

    for i in range(L-1):
        
        U, S, Vh = np.linalg.svd(psi_matrix, full_matrices=False)
        
        chi = min(k, len(S))
        
        S_truncated = S[: chi]
        U_truncated = U[:, :chi]
        Vh_truncated = Vh[:chi, :]

        mps_tensors.append(U_truncated.reshape((-1, 2, chi)))

        if i < L - 2:
            psi_matrix = (np.diag(S_truncated) @ Vh_truncated).reshape(2*chi, -1)
            
        else:
            last_tensor = (np.diag(S_truncated) @ Vh_truncated)
            mps_tensors.append(last_tensor.reshape(chi, 2, -1))

    return mps_tensors

def contract_mps_tensors(mps_tensors):

    # converts mps tensor form to a usual column vector wave-function.
    
    state = mps_tensors[0]
    
    for tensor in mps_tensors[1:]: 
        state = np.einsum('ijk,kln->ijln', state, tensor).reshape(1, -1, tensor.shape[2])
        
    return state.reshape(-1)

## Hamiltonian

In [127]:
def denseH(L, J, hx, hz, periodic):
    """
    generates the dense Hamiltonian matrix for the quantum Ising chain with both transverse and longitudinal magnetic fields
    
        Parameters:
            L (int): length of chain
            J (float): ising interaction strength
            h (float): magnetic field strength
            periodic (bool): does the chain have periodic boundary conditions?
            
        Returns:
            H (ndarray): 2^L x 2^L matrix representing the Hamiltonian operator
    """

    dim=2 ** L # dimensions of the Hilbert space
    
    H = np.zeros((dim, dim)) # initliaze the Hamiltonian
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1,L+1): # iterate over all sites
            
            alpha = beta ^ (1<<j-1) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            H[alpha, beta] -= hx # contribution by sigma^j_x
            
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim): # iterate over all states
        
        for j in range(1, L): # iterate over all sites
            
            if 2*(alpha & (1 << j-1)) == alpha & (1 << j): # check if site j and j+1 have the same spin
                
                H[alpha, alpha] -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                H[alpha, alpha] += J # if not, increase the energy by the ising interaction term
            
        "Diagonal elements due to longitudinal magnetic field"
        
        for j in range(1, L+1):
            
            if alpha & (1 << (j-1)) == 0:  # check if the spin at site j is up
                H[alpha, alpha] -= hz  # decrease energy for spin up
                
            else:
                H[alpha, alpha] += hz  # increase energy for spin down

        
        "Handling case of periodic boundary conditions"
                
        if periodic and L > 1: # L > 1 needed for periodicity to mean anything
            
            if (alpha & (1 << L-1)) == ((alpha & (1 << 0))*(2**(L-1))): # Check if the states at either end have the same spin
                
                H[alpha, alpha] -= J # if they do, decrease the energy by the ising interaction term
                
            else:
                
                H[alpha, alpha] += J # if not, increase the energy by the ising interaction term
                
    return H   

## Inner product of MPS

In [138]:
def inner_product(bra, ket):

    # function to output <psi|psi> when given in MPS form 
    
    result = np.einsum('jkl,mkn->ln', bra[0].conj(), ket[0])
    
    for i in range(1, len(ket)):

        A = np.einsum('jkl,mkn->jmln', bra[i].conj(), ket[i])
        result = np.einsum('jm,jmln->ln', result, A)
    
    return result.reshape(-1)[0]

## Function to calculate expectation value of Energy in MPS

In [169]:
def magnetic_term(ket, hx, hz):
    
    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])
    
    L = len(ket)
    results = np.zeros(L, dtype = complex)
    
    for i in range(L):
        
        Hket = [np.copy(k) for k in ket]
    
        Hket[i] = hx*np.einsum('ij,kjm->kim', X, ket[i]) + hz*np.einsum('ij,kjm->kim', Z, ket[i])
        
        results[i] = inner_product(ket, Hket)
        
    return np.sum(results)
        

def ising_term(ket):
    
    Z = np.array([[1,0],[0,-1]])
    L = len(ket)
    results = np.zeros(L, dtype = complex)
    
    for i in range(0, L-1):
        
        Hket = [np.copy(k) for k in ket]
        
        Hket[i] = np.einsum('ij,kjm->kim', Z, ket[i])
        
        Hket[i+1] = np.einsum('ij,kjm->kim', Z, ket[i+1])
        
        results[i] = inner_product(ket, Hket)
        
    return np.sum(results)

def expectation_value_energy(ket, hx, hz):
    
    energy = -magnetic_term(ket, hx, hz)-J*ising_term(ket)
    norm = inner_product(ket, ket)
    
    return energy/norm

In [200]:
L = 12
psi0 = np.array([[1], [0]])
psi = psi0
for i in range(L-1):
    psi = np.kron(psi, psi0)

print(expectation_value_energy(mps(psi, 1), -1.05, 0.5))

(-17+0j)


In [196]:
L = 12
J = 1
hx = -1.05
hz = 0.5
periodic = False 

tensors = ferromagnetic_state_MPS(L)
H = denseH(L, J, hx, hz, periodic)
eigs, vecs = scipy.linalg.eigh(H)

print('Energy of ground state: ', eigs[0])
print('Energy of the trial state: ', expectation_value_energy(tensors, hx, hz))

Energy of ground state:  -19.945778039039247
Energy of the trial state:  (-17+0j)


In [198]:
def local_gate(dt, hx, hz, ket):

    L = len(ket)

    w = np.array([[-hz, -hx], [-hx, hz]])
    R = scipy.linalg.expm(-dt*w)

    ket_new = np.zeros_like(ket)
    
    for i in range(L):
   
        ket_new[i] = np.einsum('ij,kjm->kim', R, ket[i])

    return ket_new

# def twosite_gate(dt, J, ket):

    
contract_mps_tensors(local_gate(1, 1, 1, tensors))

array([3.95889287e+06+0.j, 1.52741449e+06+0.j, 1.52741449e+06+0.j, ...,
       1.11631940e+02+0.j, 1.11631940e+02+0.j, 4.30696782e+01+0.j])

In [201]:
scipy.linalg.expm(-1*H) @ psi

array([[ 2.29864961e+08],
       [-7.30581184e+07],
       [-4.91281558e+07],
       ...,
       [-3.34651987e+03],
       [-3.93979666e+03],
       [ 2.51557811e+03]])

In [72]:
print(np.zeros_like([0, 1, 3]))

[0 0 0]


In [ ]:


def twosite_gate(dt, J, ket):

def trotter_step(dt, ket):

def bond_dimension_conserver(k, ket):
    